#### LangChain 필요성

- OpenAI API 직접 사용시 
    - 사용자 질문 -> LLM 호출 -> 응답

- 실제 서비스에서 필요한 기능
    - PDF 문서 안에서 답 찾기
    - DB 조회 후 결과 설명
    - 이전 대화 기억
    - 여러 단계 추론

#### LangChain 핵심 개념
- Chain : 여러 단계를 연결한 처리 흐름
    - ex) 검색 + 요약 + 생성 같은 다단계 흐름 구성 가능
- Prompt Template : 프롬프트를 템플릿화
- Memory : 이전 대화 저장
- Retriever (검색기)
- Agent : LLM 이 스스로 도구를 선택하여 실행

#### 활용 가능 분야
- 문서기반 QA
    - ex) 사내문서 QA 챗봇, PDF 기반 질의응답, 법률 문서 검색, 기술 매뉴얼 검색
- 챗봇 시스템
- 데이터 분석 AI
- 에이전트 기반 자동화 시스템

### RAG(Retrieval Augmented Generation)
- 검색 증강 생성
- 특정 자료(문서, PDF, DB 등)에 대해 질문에 답할 수 있다.
- 2가지 방식 제공
    - RAG Agent 방식 : Agent 를 이용해 여러번 검색이 가능하고 복잡한 질문 처리 가능
    - Two-step RAG Chain : 단순한 형태 / 한 번 호출

- 처리 단계
    - 1) Indexing : 데이터를 어떤 소스로부터 가져와서 검색할 수 있도록 정리(PDF->텍스트 추출->분할->임베딩->벡터DB 저장)
    - 2) Retrieval and Generation : 실제 RAG가 동작하는 단계(사용자가 질문을 입력하면 인덱스에서 관련 데이터를 검색하고 그 데이터를 모델에 전달하여 답변을 생성)

#### 정리
- LLM 기반 기능을 체계적으로 사용할 수 있도록 제공되는 오픈소스 프레임워크
- 외부 데이터 연결, 문서검색, 도구사용(API 호출), 멀티스텝 추론, 메모리 유지 대화 같은 복합적인 AI 애플리케이션 구조를 쉽게 만들 수 있도록 도와주는 도구

<img src="https://xeblog.s3.ap-northeast-2.amazonaws.com/langchain_builder_9f58224358.jpg" width="700" height="604">

In [ ]:
# !pip install langchain_openai

  Using cached tiktoken-0.12.0-cp314-cp314-win_amd64.whl.metadata (6.9 kB)
Using cached tiktoken-0.12.0-cp314-cp314-win_amd64.whl (921 kB)

   ------------- -------------------------- 1/3 [tiktoken]
   -------------------------- ------------- 2/3 [langchain_openai]
   -------------------------- ------------- 2/3 [langchain_openai]
   ---------------------------------------- 3/3 [langchain_openai]




[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from dotenv import load_dotenv, find_dotenv
import bs4
from pathlib import Path

load_dotenv(find_dotenv())

True

In [1]:
# 랭체인에서 제공하는 로더 
from langchain_community.document_loaders import YoutubeLoader, WebBaseLoader, PyPDFLoader 

# 임베딩 처리
from langchain_community.vectorstores import FAISS

# 문서 내용 임베딩
from langchain_openai import OpenAIEmbeddings
# 모델 생성
from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model

# 메세지
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# 프롬프트 작성
from langchain_core.prompts import PromptTemplate 

# 글을 쪼갤 때 사용하는 라이브러리
from langchain_text_splitters  import RecursiveCharacterTextSplitter

# Tools
from langchain.tools import tool
from langchain.agents import create_agent

# 크롤링
from langchain_community.document_loaders.firecrawl import FireCrawlLoader

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

c:\source\pythonsource\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
# 모델 생성
# 방법 1. 
# model = init_chat_model(model="gpt-4.1-mini", temperature=0, max_tokes=500, timeout=60)

# 방법 2
model = ChatOpenAI(model="gpt-4.1-mini")

##### Langchain
- 모델 호출
    - 1) invoke() : 모델이 답변을 전체 생성 후 응답
    - 2) stream() : 모델이 생성한 답변을 중간중간 응답(논문요약, 보고서 생성, 코드 생성, 긴 설명)
    - 3) batch() : 여러 개 요청을 한 번에 처리(병렬 처리 가능 - 리뷰 100 개 요약, 문단 50개 번역..)

In [5]:
# 모델 호출 : invoke()
# 하나의 메시지 입력 시 

res = model.invoke("사과의 효능 설명")
print(res)
print(res.usage_metadata)
print(res.content)

content='사과는 건강에 매우 유익한 과일로 다양한 효능을 가지고 있습니다. 주요 효능은 다음과 같습니다:\n\n1. **풍부한 영양소**  \n   사과에는 비타민 C, 비타민 A, 칼륨, 식이섬유 등이 풍부하게 들어 있어 전반적인 건강 유지에 도움을 줍니다.\n\n2. **소화 건강 개선**  \n   사과에 포함된 식이섬유, 특히 펙틴 성분은 장내 유익균의 먹이가 되어 장 건강을 증진시키고 변비 예방에 효과적입니다.\n\n3. **체중 관리**  \n   저칼로리에 포만감을 주는 과일로 식사 전 간식으로 섭취하면 과식을 방지하고 체중 조절에 도움이 됩니다.\n\n4. **심장 건강 증진**  \n   사과 속의 항산화 물질과 식이섬유는 혈중 콜레스테롤 수치를 낮추고 혈압을 조절하는 데 도움을 줘 심혈관 질환 예방에 도움이 됩니다.\n\n5. **항산화 작용**  \n   사과에는 플라보노이드와 폴리페놀 같은 항산화제가 풍부해 세포 손상을 막고 노화 방지에 기여합니다.\n\n6. **혈당 조절**  \n   식이섬유가 혈당 상승을 완만하게 해 당뇨병 관리에도 긍정적인 영향을 미칩니다.\n\n7. **면역력 강화**  \n   비타민 C와 같은 영양소가 면역 체계를 강화하여 감염병 예방에 도움이 됩니다.\n\n이처럼 사과는 간편하게 섭취할 수 있으면서도 여러 면에서 건강에 긍정적인 효과를 주는 과일입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 391, 'prompt_tokens': 13, 'total_tokens': 404, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio

In [7]:
# stream() : 모델이 생성한 답변을 중간중간 응답

agent = create_agent(model)

# stream_mode = "values" : 현재 상태 전체
# stream_mode = "updates" : update 내용만
for step in agent.stream({"messages":[{"role":"user","content":"사과의 효능 설명"}]}, stream_mode="values"):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

사과의 효능 설명
================================== Ai Message ==================================

사과는 맛도 좋고 건강에도 여러 가지 이로운 점이 많은 과일입니다. 사과의 주요 효능은 다음과 같습니다:

1. **항산화 작용**  
   사과에는 폴리페놀, 비타민 C 등 항산화 물질이 풍부해 체내 활성산소를 억제하고 노화 방지에 도움을 줍니다.

2. **소화 개선**  
   식이섬유가 풍부해 장 운동을 활성화시키고 변비 예방에 효과적입니다.

3. **심혈관 건강 증진**  
   사과에 들어있는 펙틴과 플라보노이드가 콜레스테롤 수치를 낮춰 주어 심장병과 고혈압 위험을 줄여줍니다.

4. **체중 관리 도움**  
   저칼로리에 식이섬유가 많아 포만감을 오래 유지시켜 다이어트에 도움이 됩니다.

5. **혈당 조절**  
   식이섬유가 혈당 상승을 완만하게 해서 당뇨 관리를 돕는 효과가 있습니다.

6. **면역력 강화**  
   비타민 C와 각종 미네랄이 면역 기능을 향상시키고 감염 예방에 도움을 줍니다.

7. **구강 건강**  
   사과는 씹는 과정에서 침 분비를 촉진해 구강 내 세균 증식을 억제하고 치아 건강에 도움을 줍니다.

이처럼 사과는 그냥 먹어도 맛있고, 다양한 건강상의 이점을 가진 과일이므로 꾸준히 섭취하는 것이 좋습니다.


In [8]:
# batch() 
inputs = [
    "사과의 효능을 설명해줘",
    "바나나의 효능을 설명해줘",
    "오렌지의 효능을 설명해줘",
]

res = model.batch(inputs)

for r in res :
    print(r.content)

사과는 건강에 유익한 다양한 성분을 포함하고 있어 여러 가지 효능이 있습니다. 대표적인 사과의 효능은 다음과 같습니다:

1. **소화 개선**  
   사과에는 식이섬유가 풍부하게 들어 있어 장 운동을 촉진하고 변비 예방에 도움을 줍니다.

2. **심혈관 건강 증진**  
   사과에 들어있는 폴리페놀과 항산화 물질은 혈관을 건강하게 유지하고 콜레스테롤 수치를 낮추어 심장 질환 예방에 효과적입니다.

3. **면역력 강화**  
   비타민 C와 항산화제가 포함되어 있어 면역 체계를 강화하고 감염에 대한 저항력을 높여줍니다.

4. **체중 관리**  
   낮은 칼로리와 풍부한 식이섬유 덕분에 포만감을 주어 다이어트에 도움이 됩니다.

5. **뇌 건강 도움**  
   사과에 포함된 항산화 물질은 뇌 세포 손상을 줄이고 인지 기능 향상에 긍정적인 영향을 미친다는 연구 결과가 있습니다.

6. **혈당 조절**  
   사과의 수용성 식이섬유는 혈당 상승을 완만하게 해주어 당뇨 환자에게도 도움이 될 수 있습니다.

이 외에도 사과는 피부 건강 개선이나 항염 작용 등 다양한 건강상의 이점을 가지고 있으니 꾸준히 섭취하는 것이 좋습니다.
바나나는 맛도 좋고 여러 가지 건강에 좋은 효능을 가지고 있는 과일입니다. 주요 효능을 설명해드리면 다음과 같습니다:

1. **에너지 공급**  
   바나나는 탄수화물이 풍부해 빠른 에너지 공급원으로 좋습니다. 특히 운동 전후에 섭취하면 에너지 보충에 도움이 됩니다.

2. **소화 개선**  
   바나나에는 식이섬유가 포함되어 있어 장 운동을 촉진하고 변비 예방에 효과적입니다.

3. **심장 건강**  
   바나나는 칼륨이 많이 들어 있어 혈압을 조절하고 심장 건강을 지키는 데 도움을 줍니다.

4. **기분 개선과 스트레스 감소**  
   바나나에 들어있는 트립토판 성분은 세로토닌으로 전환되어 기분을 좋게 하고 불안과 스트레스를 완화하는 데 도움을 줄 수 있습니다.

5. **면역력 강화**  
   비타민 C

- Messages
    - https://docs.langchain.com/oss/python/langchain/messages

In [9]:
system_msg = SystemMessage("You are a helpful assistant.")
human_msg = HumanMessage("Hello, how are you?")

res = model.invoke([system_msg, human_msg])
print(res.content)

Hello! I'm doing well, thank you. How can I assist you today?


In [10]:
# 여러 개의 메시지 리스트
# https://docs.langchain.com/oss/python/langchain/models#invoke
# 대화기록

messages = [
        {
            "role":"user",
            "content":"Which country was the most controversial in 2002 World Cup?"
        },
        {
            "role":"assistant",
            "content":"South Korea"
        },
        {
            "role":"user",
            "content":"Explain why that country was so controversial."
        },
    ]

res = model.invoke(messages)
print(res.content)

South Korea was considered the most controversial team in the 2002 FIFA World Cup primarily due to several contentious refereeing decisions during their matches in the knockout stages, which some fans, players, and analysts viewed as heavily favoring them.

Key reasons for the controversy include:

1. **Refereeing Decisions**: In both the Round of 16 match against Italy and the quarter-final against Spain, there were multiple questionable calls. For example:
   - Against Italy, a golden goal by Ahn Jung-hwan was initially ruled offside but then allowed to stand.
   - Several fouls and possible penalties against South Korea were not called.
   - In the Spain match, Spain had two clear goals disallowed, and some Italian and Spanish fans felt that the referees showed bias.

2. **Home Advantage and Pressure**: South Korea was co-hosting the tournament with Japan, and some argued that the home advantage influenced referees or added pressure on officials to favor the host nation.

3. **Unexp

- Tools : 앞에서 했었던 function calling 
    - https://docs.langchain.com/oss/python/langchain/tools

In [17]:
# 앞에서 했었던 function calling 
import json
import requests

model = ChatOpenAI(model="gpt-4.1-mini")

@tool
def get_weather(location, timezone):
    """지역명을 이용해 위,경도를 찾은 후, 위,경도로 해당 날씨 가져오기"""

    # 위,경도 찾기
    params = {"name":location,"count":1,"language":"ko"}
    geo = requests.get("https://geocoding-api.open-meteo.com/v1/search",params=params,timeout=30).json()
    
    results = geo.get("results")[0]

    if not results:
        return json.dumps({"error": f"{location} 위치를 찾지 못했습니다"})

    lat, lon = results.get('latitude'), results.get('longitude')
    print(f"위,경도 {lat}, {lon}")


    # 날씨 api 호출
    url = "https://api.open-meteo.com/v1/forecast"
    weather_params = {
        "latitude":lat,
        "longitude":lon,
        "current_weather":"true",
        "timezone":timezone
    }
    weather_result = requests.get(url,params=weather_params,timeout=30).json()

    current_weather = weather_result.get('current_weather')

    if not current_weather:
        return json.dumps({"error": "날씨 데이터를 가져오지 못했습니다"})

    # 받은 결과를 모델에게 돌려주기 위한 json 구조 설정
    return json.dumps({
        "location":location,
        "latitute":lat,
        "longitude":lon,
        "temperature_c": current_weather.get("temperature"),
        "windspeed_kmh":current_weather.get("windspeed"),       
        "winddirection_deg":current_weather.get("winddirection"),
        "weathercode":current_weather.get("weathercode"),
        "time":current_weather.get("time"),
    })

tools = [get_weather]

agent = create_agent(model=model, tools=tools)
res = agent.invoke({"messages":[{"role":"user","content":"Seoul 날씨 어때?"}]})
print(res['messages'])

res['messages'][-1].pretty_print()


위,경도 37.566, 126.9784
[HumanMessage(content='Seoul 날씨 어때?', additional_kwargs={}, response_metadata={}, id='4a532bcd-bd93-4756-9f67-c65bb0975344'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 70, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_a391f2cee0', 'id': 'chatcmpl-DD2BTYgCTSB3H3ypS1WK06jf1Y8jw', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c935f-6dda-7d51-8a2f-431e4b4b29cc-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Seoul', 'timezone': 'Asia/Seoul'}, 'id': 'call_uXq6HUIsmkKz1s9Mvn8jQEQH', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input

### LangChain 을 사용한 RAG
#### [실습 1] blog 를 기반으로 질의 검색

In [21]:
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    header_template={
        "User-Agent": "my-langchain-rag-app"
    },
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

# print(docs)
# 양이 큰 문서 -> 원하는 크기로 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 전체 문서를 text_splitter에서 적용한대로 잘라줘
all_splits = text_splitter.split_documents(docs)

## 검색을 어떻게 해올 것인가?
# 1단계: embedding 모델 정하기/생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# 2단계: 문서의 벡터화 / 벡터 DB 저장
vector_store = FAISS.from_documents(all_splits, embeddings)


@tool # function_calling 에 사용
def retrieve_context(query):
    """사용자 질문을 받아 벡터 검색 수행 / 관련 문서 2개 가져온 후 텍스트 형태로 반환"""

    ## (이전:openai) 질문-사용자의 답변 벡터화
    ## (langchian) similarity_search로 query에 대한 임베딩이 자동으로 생성
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join((f"Source : {doc.metadata}\nContent: {doc.page_content}")
                for doc in retrieved_docs)
    return serialized, retrieved_docs

# tools = [ 함수명 ]으로 사용
tools = [retrieve_context]

# prompt 정의
prompt = """
You have access to a tool that retrieves context from a blog post.
Use the tool to help answer user queries
"""

agent = create_agent(model, tools, system_prompt=prompt)

query = "What is task decomposition?"

for step in agent.stream({"messages":[{"role":"user", "content":"query"}]}, stream_mode="values"):
    step['messages'][-1].pretty_print()


================================ Human Message =================================

query
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_9UjzcUaqIZIIQ8EObbLOdYci)
 Call ID: call_9UjzcUaqIZIIQ8EObbLOdYci
  Args:
    query: query
================================= Tool Message =================================
Name: retrieve_context

('Source : {\'source\': \'https://lilianweng.github.io/posts/2023-06-23-agent/\'}\nContent: pytest\ndataclasses\n\nSource : {\'source\': \'https://lilianweng.github.io/posts/2023-06-23-agent/\'}\nContent: Conversatin samples:\n[\n  {\n    "role": "system",', [Document(id='2b1e6760-2a82-4a8e-9c0c-1439188e8c3b', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='pytest\ndataclasses'), Document(id='5143ffca-32f3-4ba6-a8a9-16ef20b4075f', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Conversatin samples:\n[\n  {\

#### [실습 2] PDF 기반 질의탐색

In [ ]:
# 1. 문서 로드드


In [ ]:
# 2. 텍스트 분할



In [ ]:
# 3. 벡터 스토어 생성



In [ ]:
# 4. 검색 tool 작성



In [ ]:
# 5. 모델 생성

In [ ]:
# 6. 질문 던지기



#### [실습 3] 유튜브 영상 자막 추출 후 내용요약

- YoutubeLoader 를 이용한 자막 추출
- RecursiveCharacterTextSplitter 를 이용한 일정한 크기로 자막 분할
- agent 를 이용한 LLM 호출 
- 응답

In [ ]:
# 1. 유튜브 자막 로드


In [ ]:
# 2. 문서 분할



In [ ]:
# -----------------------
# 3. Map 단계 (각 chunk 요약)
# -----------------------



In [ ]:
# -----------------------
# 4. Reduce 단계 (전체 결합 요약)
# -----------------------



#### Firecrawl
- 웹사이트를 크롤링하여 LLM(Long Library Management)에서 사용할 수 있는 데이터로 변환
- 접근 가능한 모든 하위 페이지를 크롤링하고 각 페이지에 대한 깔끔한 마크다운과 메타데이터를 제공
- https://docs.langchain.com/oss/python/integrations/document_loaders/firecrawl (api 키 발급 필요)

In [ ]:
# !pip install firecrawl-py

# from firecrawl import Firecrawl

# app = Firecrawl(api_key="fc-912e77013d054309a06641ad7a6df948")

# # Scrape a website:
# app.scrape('firecrawl.dev')


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
# mode="scrape" : 지정한 url 페이지 내용 가져오기
# WebBaseLoader()

# mode="crawl" : 하위 페이지까지 내용 크롤링
loader = FireCrawlLoader(url="https://firecrawl.dev", mode="scrape")
docs = loader.load()
print(len(docs))

loader = FireCrawlLoader(url="https://naver.com", mode="crawl", params={"limit":5})
docs = loader.load()
print(len(docs))

1
1


#### [실습] react 페이지 크롤링 후 크롤링 문서 기반으로 RAG 적용
- 1. Ingest(색인) 단계: URL들 → Firecrawl → Document → Split → Embedding → FAISS 저장
- 2. Query(질의) 단계: 질문 → FAISS 검색(top-k) → (질문 + 근거) → LLM 답변

In [ ]:
import re

loader = FireCrawlLoader(
    url="https://ko.react.dev/reference/react/",
    mode="crawl",
    params={
        "includePaths": [r"^/reference/react/use.*"], 
        "limit": 100,  # 넓게 크롤링
    }
)

docs = loader.load()

print(len(docs))

def get_url(doc):
    md = doc.metadata or {}
    return md.get("source_url") or md.get("source") or md.get("url") or "(unknown)"


# use~ 시작하는 url 만 걸러내기
pattern = re.compile(r"^https://ko\.react\.dev/reference/react/use.*")
use_docs = [d for d in docs if pattern.match(get_url(d))] 
print(len(use_docs))

48
19


In [9]:
# 벡터 작업
# 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 전체 문서 : text_splitter 적용 -> 분할
all_splits = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = FAISS.from_documents(all_splits, embeddings)

@tool(response_format="content_and_artifact")
def retrieve_context(query):
    """사용자 질문을 받아 벡터 검색 수행 / 관련 문서 2개 가져온 후 텍스트 형태로 반환"""

    ## query에 대한 임배딩 자동 생성
    retrieved_docs = vector_store.similarity_search(query, k=4)
    serialized = "\n\n".join((f"Source : {get_url(doc)}\nContent: {doc.page_content}")
                for doc in retrieved_docs)
    return serialized, retrieved_docs

tools = [retrieve_context]


prompt = """
너는 React 공식 문서(한국어)를 기반으로 답변하는 도우미이ㅑ.
공식 문서 안에서만 대답해. 모르면 모른다고 말해. 
답변은 한국어로, 핵심은 불릿으로 정리하고, 마지막에 출처 url 나열해. 
"""
agent = create_agent(model, tools, system_prompt=prompt)

query="What is useCallBack?"

for step in agent.stream({"messages":[{"role":"user","content":query}]}, stream_mode="values"):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

What is useCallBack?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_FS6J1DhwEB6OyflXC1vjfPwj)
 Call ID: call_FS6J1DhwEB6OyflXC1vjfPwj
  Args:
    query: useCallback
================================= Tool Message =================================
Name: retrieve_context

Source : https://ko.react.dev/reference/react/useCallback
Content: #### 주의 사항 [Link for 주의 사항 ](https://ko.react.dev/reference/react/useCallback\#caveats "Link for 주의 사항 ")

- `useCallback`은 Hook이므로, **컴포넌트의 최상위 레벨** 또는 커스텀 Hook에서만 호출할 수 있습니다. 반복문이나 조건문 내에서 호출할 수 없습니다. 이 작업이 필요하다면 새로운 컴포넌트로 분리해서 state를 새 컴포넌트로 옮기세요.
- React는 **특별한 이유가 없는 한 캐시 된 함수를 삭제하지 않습니다.** 예를 들어 개발 환경에서는 컴포넌트 파일을 편집할 때 React가 캐시를 삭제합니다. 개발 환경과 프로덕션 환경 모두에서, 초기 마운트 중에 컴포넌트가 일시 중단되면 React는 캐시를 삭제합니다. 앞으로 React는 캐시 삭제를 활용하는 더 많은 기능을 추가할 수 있습니다. 예를 들어, React에 가상화된 목록에 대한 빌트인 지원이 추가한다면, 가상화된 테이블 뷰포트에서 스크